### creating the memory features

In [1]:
import os
print("✅ key loaded:", os.getenv("GROQ_API_KEY")[:8], "...OK")


✅ key loaded: gsk_zeOc ...OK


In [2]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / "src"))
print("sys.path OK:", sys.path[0])


sys.path OK: /home/sagemaker-user/RL_2025/src


In [3]:
# Verify exactly which modules Python sees
import inspect
import backend.state, backend.rl_agent, backend.reward, backend.retrieval_policies
print(inspect.getfile(backend.state))
print(inspect.getfile(backend.rl_agent))


/home/sagemaker-user/RL_2025/src/backend/state.py
/home/sagemaker-user/RL_2025/src/backend/rl_agent.py


In [4]:
from backend.state import encode_state
from backend.rl_agent import RLAgent
from backend.reward import feedback_reward
from backend.retrieval_policies import action_to_filter

s = encode_state("How can I reduce Scope 3 emissions cost next year?", {"name":"Acme", "sector":"energy", "size":"large"})
agent = RLAgent()
a = agent.select(s)
f = action_to_filter(a, company_name="Acme")
print("state:", s)
print("action:", a, "-> filter:", f)
agent.update(s, a, feedback_reward("up"))
print("OK")


state: {'topic': 'fin', 'len': 'short', 'sector': 'energy', 'size': 'large', 'month': '2025-10'}
action: broad -> filter: None
OK


### defining the class of the assistant

In [2]:
# class ghg_assistant
# atributes: model parameteres (model, temperature, max completition tokens, system_config)
# methods:
# - generates response (user_prompt, context)
# - evaluate user prompt (if the question has any relation to the topic)
# - is a valid question (evaluate whether the question is related to GHG topic) TBD
from os import getenv
from json import loads
from json import JSONDecodeError
from groq import Client

class GHGAssistant():
    def __init__(
        self,
        system_config : str, # the definition the of task
        disclaimer : str, # the disclaimer for outputs
        model : str = 'llama-3.3-70b-versatile',
        temperature : float = 0.5,
        max_completion_tokens : int = 600,
    ):
        self.system_config, self.model, self.temp, self.max_tokens = system_config, model, temperature, max_completion_tokens
        self.disclaimer = disclaimer
        self.conversation = [] # placeholder for conversation
    def generate_response(
        self,
        user_prompt : str,
        context : str = None
    ):
        client = Client(
            api_key = getenv('GROQ_API_KEY')
        )
        # initialize the conversation
        self.conversation.append(
            # configuration of the response
            {
                'role' : 'system',
                'content' : self.system_config + f"\n\nuse the following context to answer the users question: '{context}'"
            }
        )
        self.conversation.append(
            # adding the query from the user
            {
                'role' : 'user',
                'content' : user_prompt
            }
        )
        # generating the response
        response = client.chat.completions.create(
            messages = self.conversation,
            model = self.model,
            temperature = self.temp,
            max_completion_tokens = self.max_tokens
        )
        # retreiving the output
        ai_ouput = response.choices[0].message.content
        # this section is based on the assumption that the system config asks for a json file
        try:
            json_format_ai_output = loads(ai_ouput)
            answer = json_format_ai_output.get('response', 'It was not possible to process that prompt')
            category = json_format_ai_output.get('category', 'general')
        except JSONDecodeError: # if the model output cannot be converted into 
            answer, category = ai_ouput, 'general'
        # in case the user asks for a sensitive topic
        if category in ['legal', 'financial']:
            answer += self.disclaimer
        # update conversation
        self.conversation.append(
            {
                'role' : 'assistant',
                'content' : answer
            }
        )
        return answer
    
# this is a key parameter for the class
SYSTEM_PROMPT_CONFIG = """You are a helpful AI assistant that provides your answers in as a JSON object following the structure:
{
    "response" : your answer to the user,
    "category" : your classification of the user prompt from the following list: ['legal', 'financial', 'others']
}
"""
# this can vary depending on the topic
DISCLAIMER_MESSAGE = (
    "\n\n**Disclaimer:** Be mindful that this is an AI assistant. "
    "Please consult with a professional before proceeding."
)
test_assistant = GHGAssistant(
    system_config = SYSTEM_PROMPT_CONFIG,
    disclaimer = DISCLAIMER_MESSAGE
)

test how responses differ with and without context

In [3]:
no_context_response = test_assistant.generate_response(
    user_prompt = 'what is the most popular player from colombia currently?'
)
no_context_response

'The most popular player from Colombia currently is James Rodríguez. He is a Colombian professional footballer who plays as an attacking midfielder for Al-Rayyan and the Colombia national team. However, other notable players like Radamel Falcao and Juan Cuadrado are also widely recognized.'

In [4]:
context_response = test_assistant.generate_response(
    user_prompt = 'what is the most popular player from colombia currently?',
    context = 'In the lastest match between Colombia and Brazil, Luis Diaz score two goals and secure the win for the caferos in a critical stage for the world cup'
)
context_response

'Luis Diaz is currently one of the most popular players from Colombia, known for his impressive skills and recent performances, including scoring crucial goals for the national team.'

Effectively, once the context is added the responses vary accordingly

In [5]:
test_assistant.conversation

[{'role': 'system',
  'content': 'You are a helpful AI assistant that provides your answers in as a JSON object following the structure:\n{\n    "response" : your answer to the user,\n    "category" : your classification of the user prompt from the following list: [\'legal\', \'financial\', \'others\']\n}\n\n\nuse the following context to answer the users question: \'None\''},
 {'role': 'user',
  'content': 'what is the most popular player from colombia currently?'},
 {'role': 'assistant',
  'content': 'The most popular player from Colombia currently is James Rodríguez. He is a Colombian professional footballer who plays as an attacking midfielder for Al-Rayyan and the Colombia national team. However, other notable players like Radamel Falcao and Juan Cuadrado are also widely recognized.'},
 {'role': 'system',
  'content': 'You are a helpful AI assistant that provides your answers in as a JSON object following the structure:\n{\n    "response" : your answer to the user,\n    "category"

also, the conversation is stored as an atribute of the class

In [6]:
legal_response = test_assistant.generate_response(
    user_prompt = 'how can i sue my doctor for malpractice during an operation?'
)
print(legal_response, end = "")

To sue your doctor for malpractice during an operation, you should first consult with a medical malpractice attorney who can guide you through the process. You will need to provide detailed documentation of the operation, including medical records and testimony from expert witnesses. You must also prove that the doctor's actions deviated from the standard of care and caused you harm. It's essential to act quickly, as there are statutes of limitations that vary by state. Your attorney can help you navigate the complex legal process and build a strong case.

**Disclaimer:** Be mindful that this is an AI assistant. Please consult with a professional before proceeding.

In [7]:
test_assistant.conversation

[{'role': 'system',
  'content': 'You are a helpful AI assistant that provides your answers in as a JSON object following the structure:\n{\n    "response" : your answer to the user,\n    "category" : your classification of the user prompt from the following list: [\'legal\', \'financial\', \'others\']\n}\n\n\nuse the following context to answer the users question: \'None\''},
 {'role': 'user',
  'content': 'what is the most popular player from colombia currently?'},
 {'role': 'assistant',
  'content': 'The most popular player from Colombia currently is James Rodríguez. He is a Colombian professional footballer who plays as an attacking midfielder for Al-Rayyan and the Colombia national team. However, other notable players like Radamel Falcao and Juan Cuadrado are also widely recognized.'},
 {'role': 'system',
  'content': 'You are a helpful AI assistant that provides your answers in as a JSON object following the structure:\n{\n    "response" : your answer to the user,\n    "category"